In [1]:
import pandas as pd
import numpy as np
import os
import pandas_datareader as pdr
from datetime import datetime

In [2]:
# set time frame
timeframe = '1D'

# start and end date
start_date = datetime(2004,6,1)
end_date = datetime(2009,6,1)

# set tickers
tickers = ['DIS']

# pulling data from pdr 
tickers_df = pdr.DataReader(tickers,'yahoo',start_date,end_date)

In [3]:
tickers_df.head()

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,DIS,DIS,DIS,DIS,DIS,DIS
Date,,,,,,
2004-06-01,19.184828,23.518160,23.616810,23.113695,23.478701,7336138.0
2004-06-02,19.651575,24.090330,24.119925,23.064369,23.626675,8753472.0
2004-06-03,19.289444,23.646404,24.188980,23.636539,23.922625,6509478.0
2004-06-04,19.361872,23.735189,24.050871,23.685865,23.823975,4915560.0
2004-06-07,19.635481,24.070601,24.188980,23.656269,23.922625,6578611.0


In [4]:
# closing prices dataframe
data = pd.DataFrame()

# get closing prices for all tickers in tickers
for ticker in tickers:
    data[ticker]=tickers_df['Adj Close'][ticker]

In [5]:
# add percent change
data['daily_return'] = data['DIS'].pct_change().dropna()
data['volume'] = tickers_df['Volume'][ticker]

In [6]:
data = data.dropna()
data.head()

,DIS,daily_return,volume
Date,,,
2004-06-02,19.651575,0.024329,8753472.0
2004-06-03,19.289444,-0.018428,6509478.0
2004-06-04,19.361872,0.003755,4915560.0
2004-06-07,19.635481,0.014131,6578611.0
2004-06-08,19.796427,0.008197,5967157.0


In [7]:
# EWMA Short and Long Windows
short_window = 5
long_window = 50

# Fast and Slow EWMAs from short and long windows
data['ewma_short'] = data['DIS'].ewm(halflife=short_window).mean()
data['ewma_long'] = data['DIS'].ewm(halflife=long_window).mean()

# Construct a crossover trading signal
data['crossover_long'] = np.where(data['ewma_short'] > data['ewma_long'], 1.0, 0.0) 

# making sep columns if (where) fast passes close == long 
data['crossover_short'] = np.where(data['ewma_short'] < data['ewma_long'], -1.0, 0.0)
data['crossover_signal'] = data['crossover_long'] + data['crossover_short']


In [8]:
# Volatility - Short and Long Windows
short_vol_window = 5
long_vol_window = 50

# Construct a `Fast` and `Slow` Exponential Moving Average from short and long windows, respectively
data['short_vol'] = data['daily_return'].ewm(halflife=short_vol_window).std()
data['long_vol'] = data['daily_return'].ewm(halflife=long_vol_window).std()

# Construct a crossover trading signal
data['vol_trend_long'] = np.where(data['short_vol'] < data['long_vol'], 1.0, 0.0)
data['vol_trend_short'] = np.where(data['short_vol'] > data['long_vol'], -1.0, 0.0) 
data['vol_trend_signal'] = data['vol_trend_long'] + data['vol_trend_short']


In [9]:
data = data.dropna()

In [10]:
# Set bollinger band window
bollinger_window = 20

# Calculate rolling mean and standard deviation
data['bollinger_mid_band'] = data['DIS'].rolling(window=bollinger_window).mean()
data['bollinger_std'] = data['DIS'].rolling(window=20).std() # will use for upper and lower bands of bollinger

# Calculate upper and lowers bands of bollinger band
data['bollinger_upper_band']  = data['bollinger_mid_band'] + (data['bollinger_std'] * 1)
data['bollinger_lower_band']  = data['bollinger_mid_band'] - (data['bollinger_std'] * 1)

# Calculate bollinger band trading signal # signal to calc (doing mean revering strategy)
data['bollinger_long'] = np.where(data['DIS'] < data['bollinger_lower_band'], 1.0, 0.0)
data['bollinger_short'] = np.where(data['DIS'] > data['bollinger_upper_band'], -1.0, 0.0)
data['bollinger_signal'] = data['bollinger_long'] + data['bollinger_short']

In [11]:
#Calculate simple moving average of volume
data['Short_Volume'] = data['volume'].transform(lambda x: x.rolling(window = 5).mean())
data['Long_Volume'] = data['volume'].transform(lambda x: x.rolling(window = 15).mean())
data['SMA_Volume_Ratio'] = data['Short_Volume']/data['Long_Volume']

# Construct a crossover trading signal
data['crossover_vol_long'] = np.where(data['Short_Volume'] > data['Long_Volume'], 1.0, 0.0) 

# making sep columns if (where) fast passes close == long 
data['crossover_vol_short'] = np.where(data['Short_Volume'] < data['Long_Volume'], -1.0, 0.0)
data['crossover_vol_signal'] = data['crossover_vol_long'] + data['crossover_vol_short']

In [12]:
#Calculate the average true range and keep it in a function
def add_atr_to_dataframe (dataframe, newdf):
    newdf['ATR1'] = abs (dataframe['High'][ticker] - dataframe['Low'][ticker])
    newdf['ATR2'] = abs (dataframe['High'][ticker] - dataframe['Close'][ticker].shift())
    newdf['ATR3'] = abs (dataframe['Low'][ticker] - dataframe['Close'][ticker].shift())
    newdf['TrueRange'] = newdf[['ATR1', 'ATR2', 'ATR3']].max(axis=1)
    return newdf

#Call the average true range function
add_atr_to_dataframe(tickers_df, data)

,DIS,daily_return,volume,ewma_short,ewma_long,crossover_long,crossover_short,crossover_signal,short_vol,long_vol,...,Short_Volume,Long_Volume,SMA_Volume_Ratio,crossover_vol_long,crossover_vol_short,crossover_vol_signal,ATR1,ATR2,ATR3,TrueRange
Date,,,,,,,,,,,,,,,,,,,,,
2004-06-03,19.289444,-0.018428,6509478.0,19.457979,19.469254,0.0,-1.0,-1.0,0.030233,0.030233,...,NaN,NaN,NaN,0.0,0.0,0.0,0.552441,0.098650,0.453791,0.552441
2004-06-04,19.361872,0.003755,4915560.0,19.421414,19.432963,0.0,-1.0,-1.0,0.020670,0.021311,...,NaN,NaN,NaN,0.0,0.0,0.0,0.365005,0.404467,0.039461,0.404467
2004-06-07,19.635481,0.014131,6578611.0,19.486516,19.484650,1.0,0.0,1.0,0.017443,0.018204,...,NaN,NaN,NaN,0.0,0.0,0.0,0.532711,0.453791,0.078920,0.532711
2004-06-08,19.796427,0.008197,5967157.0,19.566752,19.548746,1.0,0.0,1.0,0.014594,0.015743,...,NaN,NaN,NaN,0.0,0.0,0.0,0.276220,0.207165,0.069056,0.276220
2004-06-09,19.780334,-0.000813,7252712.0,19.615711,19.588694,1.0,0.0,1.0,0.013013,0.014346,...,6244703.6,NaN,NaN,0.0,0.0,0.0,0.394600,0.266354,0.128246,0.394600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-05-26,21.131382,0.035443,16299800.0,20.284801,18.947272,1.0,0.0,1.0,0.029966,0.036123,...,12492360.0,1.717935e+07,0.727173,0.0,-1.0,-1.0,1.230001,0.910000,0.320002,1.230001
2009-05-27,20.554443,-0.027302,14981300.0,20.319706,18.969398,1.0,0.0,1.0,0.030369,0.036041,...,13107340.0,1.705764e+07,0.768415,0.0,-1.0,-1.0,0.949999,0.179998,0.770000,0.949999
2009-05-28,20.657774,0.005027,12314700.0,20.363468,18.992643,1.0,0.0,1.0,0.028351,0.035794,...,13465440.0,1.456015e+07,0.924815,0.0,-1.0,-1.0,0.910000,0.529999,0.380001,0.910000


In [13]:
# Construct a crossover trading signal
data['crossover_atr_entry'] = np.where(data['TrueRange'] < 0.7, 1.0, 0.0) 

# making sep columns if (where) fast passes close == long 
data['crossover_atr_exit'] = np.where(data['TrueRange'] > 0.9, -1.0, 0.0)
data['crossover_atr_signal'] = data['crossover_atr_entry'] + data['crossover_atr_exit']

data

,DIS,daily_return,volume,ewma_short,ewma_long,crossover_long,crossover_short,crossover_signal,short_vol,long_vol,...,crossover_vol_long,crossover_vol_short,crossover_vol_signal,ATR1,ATR2,ATR3,TrueRange,crossover_atr_entry,crossover_atr_exit,crossover_atr_signal
Date,,,,,,,,,,,,,,,,,,,,,
2004-06-03,19.289444,-0.018428,6509478.0,19.457979,19.469254,0.0,-1.0,-1.0,0.030233,0.030233,...,0.0,0.0,0.0,0.552441,0.098650,0.453791,0.552441,1.0,0.0,1.0
2004-06-04,19.361872,0.003755,4915560.0,19.421414,19.432963,0.0,-1.0,-1.0,0.020670,0.021311,...,0.0,0.0,0.0,0.365005,0.404467,0.039461,0.404467,1.0,0.0,1.0
2004-06-07,19.635481,0.014131,6578611.0,19.486516,19.484650,1.0,0.0,1.0,0.017443,0.018204,...,0.0,0.0,0.0,0.532711,0.453791,0.078920,0.532711,1.0,0.0,1.0
2004-06-08,19.796427,0.008197,5967157.0,19.566752,19.548746,1.0,0.0,1.0,0.014594,0.015743,...,0.0,0.0,0.0,0.276220,0.207165,0.069056,0.276220,1.0,0.0,1.0
2004-06-09,19.780334,-0.000813,7252712.0,19.615711,19.588694,1.0,0.0,1.0,0.013013,0.014346,...,0.0,0.0,0.0,0.394600,0.266354,0.128246,0.394600,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-05-26,21.131382,0.035443,16299800.0,20.284801,18.947272,1.0,0.0,1.0,0.029966,0.036123,...,0.0,-1.0,-1.0,1.230001,0.910000,0.320002,1.230001,0.0,-1.0,-1.0
2009-05-27,20.554443,-0.027302,14981300.0,20.319706,18.969398,1.0,0.0,1.0,0.030369,0.036041,...,0.0,-1.0,-1.0,0.949999,0.179998,0.770000,0.949999,0.0,-1.0,-1.0
2009-05-28,20.657774,0.005027,12314700.0,20.363468,18.992643,1.0,0.0,1.0,0.028351,0.035794,...,0.0,-1.0,-1.0,0.910000,0.529999,0.380001,0.910000,0.0,-1.0,-1.0


In [14]:
data.to_csv('../Data/Disney_Closing_2008.csv',index=True)